# Imports

In [60]:
import numpy as np
import geopandas as gpd
import tensorflow as tf
import tensorflow.keras as keras
import libpysal.weights as weights
import pysal.explore as esda

# Set random seed

In [61]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [62]:
raw_df = gpd.read_file("datasets/3_combined/df.gpkg")

# Separate features

In [63]:
features = raw_df.copy()
features["x_coord"] = features["geometry"].centroid.x
features["y_coord"] = features["geometry"].centroid.y
features = features.drop(columns=["geometry", "total"])
labels = features.pop("very_good_health")

# Build model

In [64]:
# Build model
normaliser = keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(features))

model = keras.Sequential([
    normaliser,
    keras.layers.Dense(372, activation="relu"),
    keras.layers.Dense(208, activation="relu"),
    keras.layers.Dense(173, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.015686), loss="mae"
)

# Fit model
early_stopper = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True
)

model.fit(
    features,
    labels,
    batch_size=30,
    epochs=200,
    validation_split=0.2,
    callbacks=[early_stopper],
    verbose=1,
)

Epoch 1/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4192 - val_loss: 0.0533
Epoch 2/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0364 - val_loss: 0.0478
Epoch 3/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0337 - val_loss: 0.0309
Epoch 4/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0299 - val_loss: 0.0335
Epoch 5/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0304 - val_loss: 0.0344
Epoch 6/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0287 - val_loss: 0.0312
Epoch 7/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0288 - val_loss: 0.0336
Epoch 8/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0249 - val_loss: 0.0291
Epoch 9/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0263 - val_loss: 0.0316
Epoch 10/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0233 - val_loss: 0.0277
Epoch 11/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0230 - val_loss: 0.0272
Epoch 12/200
125/125 ━━━━━━━━━━━━━━━━━━━━

# Calculate spatial autocorrelation of residuals

In [65]:
predictions = model.predict(features).flatten()
residuals = labels - predictions
features["residuals"] = residuals
w = weights.KNN.from_dataframe(raw_df, k=8)
moran = esda.esda.Moran(features["residuals"], w)

146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [66]:
moran.I

np.float64(0.1676899169013092)